In [14]:
from bs4 import BeautifulSoup
import sys
import requests as req
import io

In [2]:
import pandas as pd
import html5lib
import re
import pyodbc
from datetime import datetime
import logging

In [5]:
df = pd.read_html("https://www.sport-express.ru/football/L/foreign/spain/laleague/2019-2020/")

In [523]:
#resp = req.get("https://www.sport-express.ru/football/L/foreign/spain/laleague/2019-2020/") 
resp = req.get("https://www.sport-express.ru/football/L/foreign/greece/natinal/2019-2020/") 
soup = BeautifulSoup(resp.text, 'lxml')

In [524]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- TemplateTag : football -->
  <script type="text/javascript">
   var TemplateTag = 'football';
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="a6ff411fa6eb291e" name="yandex-verification"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="kLTDoWD5oaJc9lp-iGXHnN5eyOSaTaevi-t-8lnfno8" name="google-site-verification"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76" 

In [525]:
#soup.find_all("td")
#soup.findAll("td", {"class": "se19-flex-nowrap"})
images = soup.findAll("img", {"class": "mr_10 m_all"})

In [42]:
#import image
from PIL import Image
from io import BytesIO

In [93]:
import codecs

In [526]:
image_dict = {}
for image in images:
    response = req.get('http:' + image['src'])
    image_dict[image['alt']] = Image.open(BytesIO(response.content))   

In [527]:
image_dict

{}

In [244]:
con = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',server = 'localhost', port = '1433', database = 'SportStat', autocommit=True)

In [46]:
cursor = con.cursor()
querystring = "select top 15 * from DimTeam"
result = cursor.execute(querystring)

In [47]:
result

In [51]:
# row = cursor.fetchone()
# while row:
#     print (str(row[0]) + " " + str(row[1]))
#     row = cursor.fetchone()
type(image_dict)

dict

In [ ]:
#     with open(value.load(),'rb') as f:
#         bindata = f.read()

In [54]:
image_dict.keys()

dict_keys(['Барселона', 'Реал', 'Севилья', 'Хетафе', 'Реал Сосьедад', 'Атлетико', 'Валенсия', 'Вильярреал', 'Гранада', 'Атлетик', 'Осасуна', 'Бетис', 'Леванте', 'Алавес', 'Вальядолид', 'Эйбар', 'Сельта', 'Мальорка', 'Леганес', 'Эспаньол'])

In [58]:
type(image_dict.values())

dict_values

In [107]:
#cursor = con.cursor()
for key, value in image_dict.items():
    output = io.BytesIO()
    value.save(output, format='PNG')
    tst = pyodbc.Binary(output)

In [438]:
cursor = con.cursor()
for key, value in image_dict.items():
    output = io.BytesIO()
    value.save(output, format='PNG')
    #cursor.execute("insert into DimTeam([Team], ImgSmall) values(?,?)", (str(key), pyodbc.Binary(output)))
    cursor.execute("exec pSaveTeamImg ?,?", (str(key), pyodbc.Binary(output.getvalue())))
con.commit()

In [528]:
teams = soup.findAll("span", {"class": "m_all link-underline"})

In [530]:
teams = soup.findAll("span", {"class": "m_all"})

In [531]:
teams

[<span class="m_all">26</span>,
 <span class="m_all">20</span>,
 <span class="m_all">6</span>,
 <span class="m_all">0</span>,
 <span class="m_all">53 - 9</span>,
 <span class="m_all">66</span>,
 <span class="m_all"> <a href="https://www.sport-express.ru/football/L/command/3365/">Панаитоликос</a>
 </span>,
 <span class="m_all">
 <img border="0" src="https://ss.sport-express.ru/img/design/football/home.png"/>      <span class="" title="2 : 0">
 <a href="https://www.sport-express.ru/football/foreign/greece/fbl_match-olimpiakos-p-panaitolikos-307512/">
 			2 : 0
 		</a>
 </span>
 </span>,
 <span class="m_all">26</span>,
 <span class="m_all">18</span>,
 <span class="m_all">5</span>,
 <span class="m_all">3</span>,
 <span class="m_all">50 - 23</span>,
 <span class="m_all">59</span>,
 <span class="m_all"> <a href="https://www.sport-express.ru/football/L/command/152/">Ксанти</a>
 </span>,
 <span class="m_all">
 <img border="0" src="https://ss.sport-express.ru/img/design/football/guest.png"/>   

In [532]:
type(teams)

bs4.element.ResultSet

In [533]:
tags = [] 
tags.extend(i.prettify() for i in  teams if (str(i or '').find("command") > 0))

In [534]:
team_dict = {}
for tag in tags:
    ref = tag.split('<a href="')[1].split('"')[0]
    team = tag.split('<a href="')[1].split('/">')[1].split('</a>')[0].strip()
    team_dict[team] = ref

In [535]:
team_dict

{'Панаитоликос': 'https://www.sport-express.ru/football/L/command/3365/',
 'Ксанти': 'https://www.sport-express.ru/football/L/command/152/',
 'Паниониос': 'https://www.sport-express.ru/football/L/command/1091/',
 'Волос': 'https://www.sport-express.ru/football/L/command/6465/',
 'Атромитос А': 'https://www.sport-express.ru/football/L/command/805/',
 'ПАС Ламия 1964': 'https://www.sport-express.ru/football/L/command/6043/',
 'Арис': 'https://www.sport-express.ru/football/L/command/160/',
 'Лариса': 'https://www.sport-express.ru/football/L/command/816/',
 'Астерас': 'https://www.sport-express.ru/football/L/command/2543/',
 'ПАОК': 'https://www.sport-express.ru/football/L/command/121/',
 'ОФИ': 'https://www.sport-express.ru/football/L/command/1082/',
 'Панатинаикос': 'https://www.sport-express.ru/football/L/command/103/',
 'Олимпиакос П': 'https://www.sport-express.ru/football/L/command/95/',
 'АЕК': 'https://www.sport-express.ru/football/L/command/116/'}

In [293]:
resp_team = req.get("
                    ") 
soup = BeautifulSoup(resp_team.text, 'lxml')

In [294]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- TemplateTag : football -->
  <script type="text/javascript">
   var TemplateTag = 'football';
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="a6ff411fa6eb291e" name="yandex-verification"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="kLTDoWD5oaJc9lp-iGXHnN5eyOSaTaevi-t-8lnfno8" name="google-site-verification"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76" 

In [295]:
inf = soup.findAll("div", {"class": "se19-team-profile__logo"})

In [296]:
inf

[<div class="se19-team-profile__logo">
 <img alt="Эйбар" src="https://ss.sport-express.ru/img/football/commands/1112.png" title="Эйбар"/>
 </div>]

In [186]:
type(inf)

bs4.element.ResultSet

In [297]:
teams_list = {}
for inf_el in inf:
    team_info = {}
    team_results = inf_el.find_all("img")
    for team_result in team_results:
        response = req.get(team_result['src'])
        team_info['BigImg'] = Image.open(BytesIO(response.content))

In [363]:
team_info

{'BigImg': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=120x120 at 0x1863AFC71C8>,
 'YearFound': '1940',
 'Color': '',
 'Stadium': 'Ipurua',
 'Coach': 'Хосе Луис Мендилибар',
 'StadiumNumber': ''}

In [364]:
detail_lst = []

In [365]:
details = soup.findAll("td", {"class": "pr_60"})

In [366]:
details

[<td class="pr_60"><b>Испания</b></td>,
 <td class="pr_60"><b>1940</b></td>,
 <td class="pr_60"><b></b></td>]

In [367]:
type(details)

bs4.element.ResultSet

In [368]:
for detail in details:
    detail_lst.append(str(detail).split('b>')[1].split('</')[0])
detail_lst

['Испания', '1940', '']

In [369]:
details = soup.findAll("td", {"class": "pr_0"})

In [370]:
details

[<td class="pr_0">
 <b>
                   Ipurua
                   
                 </b>
 </td>,
 <td class="pr_0">
 <b>
                   Хосе Луис Мендилибар
                 </b>
 </td>]

In [371]:
for detail in details:
    detail_lst.append(str(detail).split('b>')[1].split('</')[0].lstrip().split('\n')[0])
detail_lst

['Испания', '1940', '', 'Ipurua', 'Хосе Луис Мендилибар']

In [372]:
details = soup.findAll("span", {"class": "se19-team-profile__last-auditory ml_10"})

In [373]:
len(details)

0

In [374]:
detail_lst.append('')

In [375]:
detail_lst

['Испания', '1940', '', 'Ipurua', 'Хосе Луис Мендилибар', '']

In [376]:
if len(details) == 0:
    detail_lst.append('')
else:
    for detail in details:
        print("a")
        detail_lst.append(str(detail).split('">')[1].split("</s")[0] or '')
detail_lst

['Испания', '1940', '', 'Ipurua', 'Хосе Луис Мендилибар', '', '']

In [377]:
team_info['YearFound'] = detail_lst[1]
team_info['Color'] = detail_lst[2]
team_info['Stadium'] = detail_lst[3]
team_info['Coach'] = detail_lst[4]
team_info['StadiumNumber'] = detail_lst[5]

In [378]:
teams_list[team_result['alt']] = team_info

In [379]:
teams_list

{'Эйбар': {'BigImg': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=120x120 at 0x1863AFC71C8>,
  'YearFound': '1940',
  'Color': '',
  'Stadium': 'Ipurua',
  'Coach': 'Хосе Луис Мендилибар',
  'StadiumNumber': ''}}

In [390]:
for team, value_dict in teams_list.items():
    for key, value in value_dict.items():
        if key == 'BigImg':
            output = io.BytesIO()
            value.save(output, format='PNG')
            BigImg = pyodbc.Binary(output.getvalue())
            continue
        if key == 'YearFound':
            YearFound = value
            continue
        if key == 'Color':
            Color = value
            continue
        if key == 'Stadium':
            Stadium = value
            continue
        if key == 'Coach':
            Coach = value
            continue
        if key == 'StadiumNumber':
            StadiumNumber = value
            continue
    cursor.execute("exec pSaveTeamImg ?,?,?,?,?,?,?,?", (str(team), BigImg, 2, YearFound, Color, Stadium, StadiumNumber, Coach))
    con.commit()


In [159]:
import ParseResultsLib

In [543]:
logger = logging.getLogger()

def parse_team_update_img(images, con, country):
    image_dict = {}
    for image in images:
        response = req.get('http:' + image['src'])
        image_dict[image['alt']] = Image.open(BytesIO(response.content))
    cursor = con.cursor()
    for key, value in image_dict.items():
        output = io.BytesIO()
        value.save(output, format='PNG')
        cursor.execute("exec pSaveTeamImg ?,?,?", (str(key), country, pyodbc.Binary(output.getvalue())))
    con.commit()

def parse_team_get_team_dict(teams):
    tags = [] 
    tags.extend(i.prettify() for i in  teams if (str(i or '').find("command") > 0))
    team_dict = {}
    for tag in tags:
        ref = tag.split('<a href="')[1].split('"')[0]
        team = tag.split('<a href="')[1].split('/">')[1].split('</a>')[0].strip()
        team_dict[team] = ref
    return team_dict

def save_team_info_to_db(teams_list, con, country):
    for team, value_dict in teams_list.items():
        BigImg = ''
        YearFound = 0
        Color = ''
        Stadium = '' 
        StadiumNumber = '' 
        Coach = ''
        for key, value in value_dict.items():
            if key == 'BigImg':
                output = io.BytesIO()
                value.save(output, format='PNG')
                BigImg = pyodbc.Binary(output.getvalue())
                continue
            if key == 'YearFound':
                YearFound = value
                continue
            if key == 'Color':
                Color = value
                continue
            if key == 'Stadium':
                Stadium = value
                continue
            if key == 'Coach':
                Coach = value
                continue
            if key == 'StadiumNumber':
                StadiumNumber = value
                continue
        cursor = con.cursor()
        cursor.execute("exec pSaveTeamImg ?,?,?,?,?,?,?,?,?", (str(team), country, BigImg, 2, YearFound, Color, Stadium, StadiumNumber, Coach))
        con.commit()
        #logger.warning(f'Update db for {str(team)}')

def parse_team_get_team_info(team, url):
    resp_team = req.get(url) 
    soup = BeautifulSoup(resp_team.text, 'lxml')
    inf = soup.findAll("div", {"class": "se19-team-profile__logo"})
    team_info = {}
    detail_lst = []
    for inf_el in inf:
        team_results = inf_el.find_all("img")
        for team_result in team_results:
            response = req.get(team_result['src'])
            team_info['BigImg'] = Image.open(BytesIO(response.content))
    details = soup.findAll("td", {"class": "pr_60"})
    for detail in details:
        try:
            detail_lst.append(str(detail).split('b>')[1].split('</')[0])
        except:
            detail_lst.append('')
    details = soup.findAll("td", {"class": "pr_0"})
    for detail in details:
        try:
            detail_lst.append(str(detail).split('b>')[1].split('</')[0].lstrip().split('\n')[0])
        except:
            detail_lst.append('')
    details = soup.findAll("span", {"class": "se19-team-profile__last-auditory ml_10"})
    if len(details) == 0:
        detail_lst.append('')
    else:
        for detail in details:
            try:
                detail_lst.append(str(detail).split('">')[1].split("</s")[0])
            except:
                detail_lst.append('')
    team_info['YearFound'] = detail_lst[1]
    team_info['Color'] = detail_lst[2]
    team_info['Stadium'] = detail_lst[3]
    team_info['Coach'] = detail_lst[4]
    team_info['StadiumNumber'] = detail_lst[5]
    return team_info
    
def parse_team_get_team_info_main(team_dict, con, IsOverwrite, country):
    teams_list = {}
    for team, url in team_dict.items():
        logger.warning(team)
        team_info = parse_team_get_team_info(team, url)
        teams_list[team] = team_info
        #logger.warning("Получены данные")
        url_lineup =url + 'players/'
        #logger.warning(f"url_lineup = {url_lineup}")
        GetTeamlineUp(con, team, url_lineup, IsOverwrite)
    logger.warning(teams_list)
    save_team_info_to_db(teams_list, con, country)
    logger.warning("Done save_team_info_to_db")
    
def parse_team_data(url, con, country, IsOverwrite = 1):
    
    #logger.warning('Start parse_team_data')
    resp = req.get(url) 
    soup = BeautifulSoup(resp.text, 'lxml')
    
    #Основная функция: принимает на вход url турнирной таблицы
    #logger.warning('Start parse_team_update_img')
    images = soup.findAll("img", {"class": "mr_10 m_all"})
    try:
        parse_team_update_img(images, con, country)
        logger.warning('Done parse_team_update_img')
    except:
        logger.error('Error parse_team_update_img')
    
    #Парсинг и сохранение в базу инфо о команде
    teams = soup.findAll("span", {"class": "m_all link-underline"})
    if len(teams) == 0:
        teams = soup.findAll("span", {"class": "m_all"})
    logger.warning(f'teams: {teams}')
    team_dict = parse_team_get_team_dict(teams)
    logger.warning(f'team_dict: {team_dict}')
    logger.warning('Done parse_team_get_team_dict')
    #logger.warning('Start parse_team_get_team_info_main')
    parse_team_get_team_info_main(team_dict, con, IsOverwrite, country)
    logger.warning('Done parse_team_get_team_info_main')
    SaveNations(con, nations)
    #logger.warning('SaveNations done')
    

def GetPlayerField(tr):
    cls = tr.attrs.get("class")
    if cls is not None and 'w_100p' in cls:
        img = tr.find("img")
        if img is not None:
            return img['title'] + '|' + img['src']
    return tr.text

def GetTeamlineUp(con, Team, url, IsOverwrite):
    resp = req.get(url) 
    soup_team = BeautifulSoup(resp.text, 'lxml')
    player_table = soup_team.find("table", {"class": "se19-table"})
    if player_table is not None:
        player_list = player_table.find('tbody').find_all('tr')

        l = []
        for tr in player_list:
            td = tr.find_all('td')
            row = [GetPlayerField(tr) for tr in td]
            #print(row)
            l.append(row)
        if len(l)!= 0:
            if len(l[0]) == 6:
                IsFull = 1
                df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua", "Games", "Goal", "Pass",])
            elif len(l[0]) == 3:
                IsFull = 0
                df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua"])
            for index, row in df_players.iterrows():
                tst_dict = {}
                Player = row['Player'].strip()
                Amplua =  row['Amplua'].strip()
                if IsFull == 1:
                    Games =  row['Games']
                    Pass =  row['Pass']    
                    if Amplua == 'В' and '(' in row['Goal']:
                        Goals =  row['Goal'].split('(')[0]
                        GoalsGK =  row['Goal'].split('(')[1].split(')')[0]
                    else:
                        Goals =  row['Goal']
                        GoalsGK = 0
                else:
                    Games = 0
                    Pass = 0
                    Goals = 0
                    GoalsGK = 0
                if '|' in row['Nation']:
                    tst_dict['URL'] = row['Nation'].split('|')[1]
                    nations[row['Nation'].split('|')[0]] = tst_dict
                cursor = con.cursor()
                cursor.execute("exec pSaveLineUp ?,?,?,?,?,?,?,?", (Team, IsOverwrite, Player, Amplua, Games, Goals, Pass, GoalsGK))
                con.commit()

def SaveNations(con, nations):   
    for nation_key, nation_value  in nations.items():
        response = req.get(nation_value['URL'])
        nations[nation_key]['Img'] = Image.open(BytesIO(response.content))
    cursor = con.cursor()
    for key, value in nations.items():
        output = io.BytesIO()
        value['Img'].save(output, format='PNG')
        cursor.execute("exec pSaveNation ?,?,?", (str(key), value['URL'], pyodbc.Binary(output.getvalue())))
    con.commit()
    
def GetUrlList(con, Country, IsMainCountry, LeagueType, IDSeasonType):
    CursorLeague = con.cursor()
    querystring = f"select CurrentSource, Country from fGetCountrySource(2, '{Country}', 1) where IsMainCountry = {IsMainCountry} and LeagueType = {LeagueType}  order by Country"
    Leagues = CursorLeague.execute(querystring)
    RowLeague = CursorLeague.fetchone()
    league_list = []

    while RowLeague:
        league_list.append(RowLeague[0])
        RowLeague = CursorLeague.fetchone()

    CursorSeason = con.cursor()
    querystring = f"select * from fGetSeason(1, NULL, {IDSeasonType})"
    ResultSeason = CursorSeason.execute(querystring)

    season_lst = []
    RowSeason = CursorSeason.fetchone()
    while RowSeason:
        season_lst.append(RowSeason[0])
        RowSeason = CursorSeason.fetchone()

    url_lst = []
    for season in season_lst:
        tmp = list(map(lambda x: x.replace('[Season]', season).replace('calendar/tours/', ''), league_list))
        url_lst+=tmp
    return url_lst


In [357]:
parse_team_get_team_info('Эйбар', 'https://www.sport-express.ru/football/L/command/1112/')

{'BigImg': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=120x120 at 0x1863D88B208>,
 'YearFound': '1940',
 'Color': '',
 'Stadium': 'Ipurua',
 'Coach': 'Хосе Луис Мендилибар',
 'StadiumNumber': ''}

In [246]:
team_dict = {'Барселона': 'https://www.sport-express.ru/football/L/command/78/'}
parse_team_get_team_info_main(team_dict)

In [404]:
url_lst = ["https://www.sport-express.ru/football/L/foreign/spain/laleague/2019-2020/",
          ]

In [464]:
con = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',server = 'localhost', port = '1433', database = 'SportStat', autocommit=True)

In [482]:
Country = 'Англия'
CursorLeague = con.cursor()
querystring = f"select CurrentSource, Country from fGetCountrySource(2, '{Country}', 1) where IsMainCountry = 1 and LeagueType = 1  order by Country"
Leagues = CursorLeague.execute(querystring)
RowLeague = CursorLeague.fetchone()
league_list = []

while RowLeague:
    league_list.append(RowLeague[0])
    RowLeague = CursorLeague.fetchone()

CursorSeason = con.cursor()
querystring = "select * from fGetSeason(1, NULL)"
ResultSeason = CursorSeason.execute(querystring)

season_lst = []
RowSeason = CursorSeason.fetchone()
while RowSeason:
    season_lst.append(RowSeason[0])
    RowSeason = CursorSeason.fetchone()

In [483]:
url_lst = []
for season in season_lst:
    tmp = list(map(lambda x: x.replace('[Season]', season).replace('calendar/tours/', ''), league_list))
    url_lst+=tmp

In [487]:
url_lst

In [494]:
url_lst = GetUrlList(con, "Англия", 1, 1)

In [503]:
nations = {}
con = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',server = 'localhost', port = '1433', database = 'SportStat', autocommit=True)
logger.warning('Start parse')

Country = 'Англия'
CursorCountry = con.cursor()
querystring = f"select Country, IsMain, IDSeasonType from DimCountry"
Countries = CursorCountry.execute(querystring)
RowCountry = CursorCountry.fetchone()
country_dict = {}

while RowCountry:
    cnt_d = {}
    Country = RowCountry[0]
    IsMain = RowCountry[1]
    IDSeasonType = RowCountry[2]
    cnt_d["IsMain"] = IsMain
    cnt_d["IDSeasonType"] = IDSeasonType
    country_dict[Country] = cnt_d
    RowCountry = CursorCountry.fetchone()

for key, value in country_dict.items():
    Country = key
    IsMain = value["IsMain"]
    IDSeasonType  = value["IDSeasonType"]
    url_lst = GetUrlList(con, Country, IsMain, 1, IDSeasonType)
    logger.warning(Country)

    for url in url_lst:
        print(url)
        parse_team_data(url, con, Country)

Start parse


http://football.sport-express.ru/foreign/england/premier/2005-2006/


Done parse_team_update_img
Done parse_team_get_team_dict
Челси
Ньюкасл Юнайтед
Манчестер Юнайтед
Чарльтон Атлетик
Ливерпуль
Портсмут
Арсенал
Уиган Атлетик
Тоттенхэм Хотспур
Вест Хэм Юнайтед
Блэкберн Роверс
Манчестер Сити
Болтон Уондерерс
Бирмингем Сити
Эвертон
Вест Бромвич Альбион
Фулхэм
Мидлсбро
Астон Вилла
Сандерленд
Done save_team_info_to_db
Done parse_team_get_team_info_main
Done parse_team_update_img
Done parse_team_get_team_dict
Done save_team_info_to_db
Done parse_team_get_team_info_main


https://www.sport-express.ru/football/L/foreign/england/champion/2005-2006/
http://football.sport-express.ru/foreign/england/premier/2006-2007/


Done parse_team_update_img
Done parse_team_get_team_dict
Манчестер Юнайтед
Вест Хэм Юнайтед
Челси
Эвертон
Ливерпуль
Чарльтон Атлетик
Арсенал
Портсмут
Тоттенхэм Хотспур
Манчестер Сити
Болтон Уондерерс
Астон Вилла
Рединг
Блэкберн Роверс
Мидлсбро
Фулхэм
Ньюкасл Юнайтед
Уотфорд
Уиган Атлетик
Шеффилд Юнайтед
Done save_team_info_to_db
Done parse_team_get_team_info_main
Done parse_team_update_img
Done parse_team_get_team_dict
Done save_team_info_to_db
Done parse_team_get_team_info_main


...

Done parse_team_update_img
Done parse_team_get_team_dict
Норвич
Астон Вилла
Шеффилд Юнайтед
Стоук Сити
Лидс
Ипсвич
Вест Бромвич Альбион
Дерби Каунти
Мидлсбро
Ротерхэм Юнайтед
Бристоль Сити
Халл Сити
Ноттингем Форест
Болтон Уондерерс
Суонси
Блэкберн Роверс
Брентфорд
Престон Норт Энд
Шеффилд Уэнсдей
Куинз Парк Рейнджерс
Бирмингем Сити
Рединг
Уиган Атлетик
Миллуолл
Done save_team_info_to_db
Done parse_team_get_team_info_main


http://football.sport-express.ru/foreign/england/premier/2019-2020/


Done parse_team_update_img
Done parse_team_get_team_dict
Ливерпуль
Борнмут
Манчестер Сити
Манчестер Юнайтед
Лестер
Астон Вилла
Челси
Эвертон
Вулверхэмптон Уондерерс
Брайтон &amp; Хоув
Шеффилд Юнайтед
Норвич
Тоттенхэм Хотспур
Бернли
Арсенал
Вест Хэм Юнайтед
Кристал Пэлас
Уотфорд
Ньюкасл Юнайтед
Саутгемптон
Done save_team_info_to_db
Done parse_team_get_team_info_main


https://www.sport-express.ru/football/L/foreign/england/champion/2019-2020/


Done parse_team_update_img
Done parse_team_get_team_dict
Лидс
Хаддерсфилд
Вест Бромвич Альбион
Суонси
Фулхэм
Бристоль Сити
Брентфорд
Шеффилд Уэнсдей
Ноттингем Форест
Миллуолл
Престон Норт Энд
Куинз Парк Рейнджерс
Кардифф Сити
Барнсли
Блэкберн Роверс
Дерби Каунти
Рединг
Бирмингем Сити
Стоук Сити
Халл Сити
Мидлсбро
Чарльтон Атлетик
Уиган Атлетик
Лутон Таун
Done save_team_info_to_db
Done parse_team_get_team_info_main


In [545]:
url_lst = ['https://sport-express.ru/football/L/foreign/bulgaria/highest/2007-2008/']    
for url in url_lst:
    print(url)
    parse_team_data(url, con, country)
        

https://sport-express.ru/football/L/foreign/bulgaria/highest/2007-2008/


Done parse_team_update_img
teams: [<span class="m_all">30</span>, <span class="m_all">24</span>, <span class="m_all">6</span>, <span class="m_all">0</span>, <span class="m_all">53 - 11</span>, <span class="m_all">78</span>, <span class="m_all"> <a href="https://www.sport-express.ru/football/L/command/1455/">Вихрен</a>
</span>, <span class="m_all">
<img border="0" src="https://ss.sport-express.ru/img/design/football/guest.png"/>      <span class="" title="2 : 1">
<a href="https://www.sport-express.ru/football/foreign/bulgaria/fbl_match-vihren-cska-sf-59972/">
			2 : 1
		</a>
</span>
</span>, <span class="m_all">30</span>, <span class="m_all">19</span>, <span class="m_all">5</span>, <span class="m_all">6</span>, <span class="m_all">56 - 19</span>, <span class="m_all">62</span>, <span class="m_all"> <a href="https://www.sport-express.ru/football/L/command/1461/">Славия Сф</a>
</span>, <span class="m_all">
<img border="0" src="https://ss.sport-express.ru/img/design/football/home.png"/>    

team_dict: {'Вихрен': 'https://www.sport-express.ru/football/L/command/1455/', 'Славия Сф': 'https://www.sport-express.ru/football/L/command/1461/', 'Берое': 'https://www.sport-express.ru/football/L/command/1457/', 'Черноморец Бр': 'https://www.sport-express.ru/football/L/command/2285/', 'Видима Раковски': 'https://www.sport-express.ru/football/L/command/2504/', 'Литекс': 'https://www.sport-express.ru/football/L/command/165/', 'Левски': 'https://www.sport-express.ru/football/L/command/107/', 'Локомотив Пл': 'https://www.sport-express.ru/football/L/command/180/', 'Пирин Б': 'https://www.sport-express.ru/football/L/command/2284/', 'ЦСКА Сф': 'https://www.sport-express.ru/football/L/command/130/', 'Марек': 'https://www.sport-express.ru/football/L/command/1460/', 'Беласица П': 'https://www.sport-express.ru/football/L/command/1462/', 'Локомотив Сф': 'https://www.sport-express.ru/football/L/command/769/', 'Ботев Пл': 'https://www.sport-express.ru/football/L/command/1458/', 'Спартак Вр': 'htt